In [21]:
#Import python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score

In [13]:
from google.colab import files
uploaded = files.upload()

Saving Data_for_UCI_named.csv to Data_for_UCI_named.csv


In [15]:
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [16]:
df.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


In [24]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [25]:
x = df.drop(columns=['stab', 'stabf'])
y= df['stabf']

In [27]:
#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [28]:
# Instantiating the Standard Scaler
sc = StandardScaler()

# fit and transform the training set
x_train = pd.DataFrame(sc.fit_transform(x_train), columns=x_train.columns)

# transfoorm the test set
x_test = pd.DataFrame(sc.transform(x_test), columns=x_test.columns)

**Question 1**; According to a use-case, in a certain ML task, a false positive is six times costlier than a false negative. You, as a Data Scientist, trained 4 models, to solve the use case.

Keep the following evaluation criteria in mind:

1) Must have a recall rate of at least 80%

2) Must have a false positive rate of 8% or less

3) Must minimize business costs

 After creating each binary classification model, you generated the corresponding confusion matrix. Which confusion matrix represents the model that satisfies the requirements

 **Answer - TN = 96%, FP = 4%, FN = 10%, TP = 90% (option B)**

**Question 2**; You are working on a spam classification system using regularized logistic regression. “Spam” is a positive class (y = 1) and “not spam” is the negative class (y = 0). You have trained your classifier and there are n = 1700 examples in the test set. The confusion matrix of predicted class vs. actual class is:
What is the F1 score of this classifier?

**Answer; Precision = 255 / (255 + 1380) = 0.155
Recall = 255 / (255 + 20) = 0.927
Now,the F1 score:
F1 score = 2 * (0.155 * 0.927) / (0.155 + 0.927) = 0.2635**

**Question 3**; Jack is working on classification modelling. While evaluating the model, he saw that the difference between test and training error is a big positive number with a low training error. Which of the following, is he currently facing?

**Answer; Overfitting**

**Question 4**; Which of the following metric is generally NOT useful for a classification problem?

**Answer; RMSE Value**

**Question 5**; Why do we use weak learners in boosting, instead if strong learners?

**Answer; To prevent overfitting**


**Question 6**; You are building a classifier and the accuracy is poor on both the training and test sets. Which would you use to try to improve the performance?

**Answer; None of the above**


**Question 7**; A classifier predicts if insurance claims are fraudulent or not. The cost of paying a fraudulent claim is higher than the cost of investigating a claim that is suspected to be fraudulent. Which metric should we use to evaluate this classifier?

**Answer; Recall**

**Question 8**; The ROC curve above was generated from a classification algorithm. What can we say about this classifier?

**Answer; The model has no discrimination capacity to differentiate between the positive and the negative class**

**Question 9**; A random forest classifier was used to classify handwritten digits 0-9 into the numbers they were intended to represent. The confusion matrix below was generated from the results. Based on the matrix, which number was predicted with the least accuracy?

**Answer; 8**

**Question 10**; A medical company is building a model to predict the occurrence of thyroid cancer. The training data contains 900 negative instances (people who don't have cancer) and 100 positive instances. The resulting model has 90% accuracy, but extremely poor recall. What steps can be used to improve the model's performance? (SELECT TWO)

**Answer; Collect more data for the positive case and
Generate synthetic samples/data using SMOTE**

**Question 11**; You are developing a machine learning classification algorithm that categorizes handwritten digits 0-9 into the numbers they represent. How should you pre-process the label data?

**Answer; One-hot encoding**


**Questiion 12**; What is the entropy of the target variable if its actual values are given as:[1,0,1,1,0,1,0]

**Answer; - 3/7 log(4/7) + 4/7 log(3/7)**

**Question 13**; Which of these is not a good metric for evaluating classification algorithms for data with imbalanced class problems?

**Answer; Accuracy**



**Question 14**; What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

**Answer; 0.9291**

In [37]:
clf = RandomForestClassifier(random_state = 1).fit(x_train, y_train)


In [44]:
# Create our predictions
prediction = clf.predict(x_test)
# Create confusion matrix
confusion_matrix(y_test, prediction)
# Display accuracy score
round(accuracy_score(y_test, prediction), 4)


0.929

**Question 15**; What is the accuracy on the test set using the XGboost classifier? In 4 decimal places.

**Answer; 0.9195**

**Question 16**; What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.
**Answer; 0.9375**

In [56]:
lgbm_clf = LGBMClassifier(random_state=1)

# Fit the classifier with training set
lgbm_clf.fit(x_train, y_train)

# make predictions
lgbm_pred = lgbm_clf.predict(x_test)

# compute the accuracy score
acc_score = round(accuracy_score(y_test, lgbm_pred), 4)
acc_score

0.9395

**Question 17**; Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV

In [58]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

# Define the param grid
hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

# Instantiate the ExtratreClassifer
ext_clf = ExtraTreesClassifier(random_state=1)

# instantiate the gridsearch
randomised_grid_search = RandomizedSearchCV(ext_clf, param_distributions= hyperparameter_grid,
                                scoring='accuracy',
                                n_iter=10,
                                cv=5,
                                n_jobs=-1,
                                verbose=1)

# fit the grid with training set
randomised_grid_search.fit(x_train, y_train)
randomised_grid_search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'n_estimators': 300,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto'}

**Question 18**; Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?

**Answer; Higher**

In [59]:
ext_clf.fit(x_train, y_train)

old_acc_score = round(accuracy_score(y_test, ext_clf.predict(x_test)), 4)
print(f'old accuracy score is {old_acc_score}')


# Get the model with new hyperparameters from the randomsearchcv
new_model = randomised_grid_search.best_estimator_

# Train a new ExtraTree model with new hyparameters
new_model.fit(x_train, y_train)

# make new predictions
new_pred = new_model.predict(x_test)

# Compute new accuracy score
print(f'New Accuracy score is {round(accuracy_score(y_test, new_pred), 4)}')


old accuracy score is 0.928


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


New Accuracy score is 0.9315


**Question 19**; What other hyperparameter optimization method can you try apart from Random Search?

Options
Evolutionary optimization

Bayesian optimization

A, B and C

Grid search

**Answer; Both A and B**

**Qustion 20**; Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least important respectively?

Options
g3, p4

tau 4, tau3

g1, tau1

**Answer; tau2, p1**